# ESCUELA POLITECNICA NACIONAL
### Nombre: Kleber Janeta

1. Obtencion de los datos

In [24]:
#Librerias a usar
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [25]:
df_critics= pd.read_csv(r"/content/rotten_tomatoes_critic_reviews.csv") # Carga de csv con las reviews
df_movies= pd.read_csv(r"/content/rotten_tomatoes_movies.csv") # Carga de csv con las movies

In [26]:
df_critics # impresion de dataframe a usarse

,rotten_tomatoes_link,critic_name,top_critic,publisher_name,review_type,review_score,review_date,review_content
0,m/0814255,Andrew L. Urban,False,Urban Cinefile,Fresh,NaN,2010-02-06,A fantasy adventure that fuses Greek mythology...
1,m/0814255,Louise Keller,False,Urban Cinefile,Fresh,NaN,2010-02-06,"Uma Thurman as Medusa, the gorgon with a coiff..."
2,m/0814255,NaN,False,FILMINK (Australia),Fresh,NaN,2010-02-09,With a top-notch cast and dazzling special eff...
3,m/0814255,Ben McEachen,False,Sunday Mail (Australia),Fresh,3.5/5,2010-02-09,Whether audiences will get behind The Lightnin...
4,m/0814255,Ethan Alter,True,Hollywood Reporter,Rotten,NaN,2010-02-10,What's really lacking in The Lightning Thief i...
...,...,...,...,...,...,...,...,...
1130012,m/zulu_dawn,Chuck O'Leary,False,Fantastica Daily,Rotten,2/5,2005-11-02,NaN
1130013,m/zulu_dawn,Ken Hanke,False,"Mountain Xpress (Asheville, NC)",Fresh,3.5/5,2007-03-07,"Seen today, it's not only a startling indictme..."
1130014,m/zulu_dawn,Dennis Schwartz,False,Dennis Schwartz Movie Reviews,Fresh,B+,2010-09-16,A rousing visual spectacle that's a prequel of...
1130015,m/zulu_dawn,Christopher Lloyd,False,Sarasota Herald-Tribune,Rotten,3.5/5,2011-02-28,"A simple two-act story: Prelude to war, and th..."


In [27]:
def preprocesamiento (text):
    if pd.isna(text): # Eliminamos los NaN y dejamos vacio
        return ""
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)  # Eliminar caracteres especiales
    text = re.sub(r"\s+", " ", text)  # Eliminar espacios extra
    return text.lower() # retorna el texto con preprocesamiento base

In [28]:
df_critics["reviewsprepros"]= df_critics["review_content"].apply(preprocesamiento) # aplicamos la funcion de preprocesamiento a la columna review_content
df_critics # imprimimos dataframe

,rotten_tomatoes_link,critic_name,top_critic,publisher_name,review_type,review_score,review_date,review_content,reviewsprepros
0,m/0814255,Andrew L. Urban,False,Urban Cinefile,Fresh,NaN,2010-02-06,A fantasy adventure that fuses Greek mythology...,a fantasy adventure that fuses greek mythology...
1,m/0814255,Louise Keller,False,Urban Cinefile,Fresh,NaN,2010-02-06,"Uma Thurman as Medusa, the gorgon with a coiff...",uma thurman as medusa the gorgon with a coiffu...
2,m/0814255,NaN,False,FILMINK (Australia),Fresh,NaN,2010-02-09,With a top-notch cast and dazzling special eff...,with a topnotch cast and dazzling special effe...
3,m/0814255,Ben McEachen,False,Sunday Mail (Australia),Fresh,3.5/5,2010-02-09,Whether audiences will get behind The Lightnin...,whether audiences will get behind the lightnin...
4,m/0814255,Ethan Alter,True,Hollywood Reporter,Rotten,NaN,2010-02-10,What's really lacking in The Lightning Thief i...,whats really lacking in the lightning thief is...
...,...,...,...,...,...,...,...,...,...
1130012,m/zulu_dawn,Chuck O'Leary,False,Fantastica Daily,Rotten,2/5,2005-11-02,NaN,
1130013,m/zulu_dawn,Ken Hanke,False,"Mountain Xpress (Asheville, NC)",Fresh,3.5/5,2007-03-07,"Seen today, it's not only a startling indictme...",seen today its not only a startling indictment...
1130014,m/zulu_dawn,Dennis Schwartz,False,Dennis Schwartz Movie Reviews,Fresh,B+,2010-09-16,A rousing visual spectacle that's a prequel of...,a rousing visual spectacle thats a prequel of ...
1130015,m/zulu_dawn,Christopher Lloyd,False,Sarasota Herald-Tribune,Rotten,3.5/5,2011-02-28,"A simple two-act story: Prelude to war, and th...",a simple twoact story prelude to war and the w...


In [29]:
pip install  nltk # intalaicon de nltk

In [30]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords') # descarga de stopwords en ingles
stop_words = set(stopwords.words('english'))  # obtencion de stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [31]:
# Función para eliminar las stopwords
def eliminar_stopwords(texto):
    if not isinstance(texto, str):  # Manejar valores vacíos o nulos
        return texto
    palabras = texto.split() #tokenizacion de palabras
    palabras_sin_stopwords = [palabra for palabra in palabras if palabra.lower() not in stop_words] # for para elimianr stpwords y colocar el texto sin stopwords en una nueva lista
    return ' '.join(palabras_sin_stopwords) # transformacion de tokens a texto

In [32]:
# Aplicar la función a la columna
df_critics['reviewssinstopwords'] = df_critics['reviewsprepros'].apply(eliminar_stopwords)

In [33]:
df_critics

,rotten_tomatoes_link,critic_name,top_critic,publisher_name,review_type,review_score,review_date,review_content,reviewsprepros,reviewssinstopwords
0,m/0814255,Andrew L. Urban,False,Urban Cinefile,Fresh,NaN,2010-02-06,A fantasy adventure that fuses Greek mythology...,a fantasy adventure that fuses greek mythology...,fantasy adventure fuses greek mythology contem...
1,m/0814255,Louise Keller,False,Urban Cinefile,Fresh,NaN,2010-02-06,"Uma Thurman as Medusa, the gorgon with a coiff...",uma thurman as medusa the gorgon with a coiffu...,uma thurman medusa gorgon coiffure writhing sn...
2,m/0814255,NaN,False,FILMINK (Australia),Fresh,NaN,2010-02-09,With a top-notch cast and dazzling special eff...,with a topnotch cast and dazzling special effe...,topnotch cast dazzling special effects tide te...
3,m/0814255,Ben McEachen,False,Sunday Mail (Australia),Fresh,3.5/5,2010-02-09,Whether audiences will get behind The Lightnin...,whether audiences will get behind the lightnin...,whether audiences get behind lightning thief h...
4,m/0814255,Ethan Alter,True,Hollywood Reporter,Rotten,NaN,2010-02-10,What's really lacking in The Lightning Thief i...,whats really lacking in the lightning thief is...,whats really lacking lightning thief genuine s...
...,...,...,...,...,...,...,...,...,...,...
1130012,m/zulu_dawn,Chuck O'Leary,False,Fantastica Daily,Rotten,2/5,2005-11-02,NaN,,
1130013,m/zulu_dawn,Ken Hanke,False,"Mountain Xpress (Asheville, NC)",Fresh,3.5/5,2007-03-07,"Seen today, it's not only a startling indictme...",seen today its not only a startling indictment...,seen today startling indictment british imperi...
1130014,m/zulu_dawn,Dennis Schwartz,False,Dennis Schwartz Movie Reviews,Fresh,B+,2010-09-16,A rousing visual spectacle that's a prequel of...,a rousing visual spectacle thats a prequel of ...,rousing visual spectacle thats prequel cy endf...
1130015,m/zulu_dawn,Christopher Lloyd,False,Sarasota Herald-Tribune,Rotten,3.5/5,2011-02-28,"A simple two-act story: Prelude to war, and th...",a simple twoact story prelude to war and the w...,simple twoact story prelude war war former con...


In [34]:
#Funcion para tokenizar y eliminar stopwords

def textoTokenizado(texto):
    palabras = texto.split()  # Dividir el texto en palabras
    return palabras

In [35]:
df_critics['textoTokenizado'] = df_critics['reviewssinstopwords'].apply(lambda texto: textoTokenizado(texto))
df_critics

,rotten_tomatoes_link,critic_name,top_critic,publisher_name,review_type,review_score,review_date,review_content,reviewsprepros,reviewssinstopwords,textoTokenizado
0,m/0814255,Andrew L. Urban,False,Urban Cinefile,Fresh,NaN,2010-02-06,A fantasy adventure that fuses Greek mythology...,a fantasy adventure that fuses greek mythology...,fantasy adventure fuses greek mythology contem...,"[fantasy, adventure, fuses, greek, mythology, ..."
1,m/0814255,Louise Keller,False,Urban Cinefile,Fresh,NaN,2010-02-06,"Uma Thurman as Medusa, the gorgon with a coiff...",uma thurman as medusa the gorgon with a coiffu...,uma thurman medusa gorgon coiffure writhing sn...,"[uma, thurman, medusa, gorgon, coiffure, writh..."
2,m/0814255,NaN,False,FILMINK (Australia),Fresh,NaN,2010-02-09,With a top-notch cast and dazzling special eff...,with a topnotch cast and dazzling special effe...,topnotch cast dazzling special effects tide te...,"[topnotch, cast, dazzling, special, effects, t..."
3,m/0814255,Ben McEachen,False,Sunday Mail (Australia),Fresh,3.5/5,2010-02-09,Whether audiences will get behind The Lightnin...,whether audiences will get behind the lightnin...,whether audiences get behind lightning thief h...,"[whether, audiences, get, behind, lightning, t..."
4,m/0814255,Ethan Alter,True,Hollywood Reporter,Rotten,NaN,2010-02-10,What's really lacking in The Lightning Thief i...,whats really lacking in the lightning thief is...,whats really lacking lightning thief genuine s...,"[whats, really, lacking, lightning, thief, gen..."
...,...,...,...,...,...,...,...,...,...,...,...
1130012,m/zulu_dawn,Chuck O'Leary,False,Fantastica Daily,Rotten,2/5,2005-11-02,NaN,,,[]
1130013,m/zulu_dawn,Ken Hanke,False,"Mountain Xpress (Asheville, NC)",Fresh,3.5/5,2007-03-07,"Seen today, it's not only a startling indictme...",seen today its not only a startling indictment...,seen today startling indictment british imperi...,"[seen, today, startling, indictment, british, ..."
1130014,m/zulu_dawn,Dennis Schwartz,False,Dennis Schwartz Movie Reviews,Fresh,B+,2010-09-16,A rousing visual spectacle that's a prequel of...,a rousing visual spectacle thats a prequel of ...,rousing visual spectacle thats prequel cy endf...,"[rousing, visual, spectacle, thats, prequel, c..."
1130015,m/zulu_dawn,Christopher Lloyd,False,Sarasota Herald-Tribune,Rotten,3.5/5,2011-02-28,"A simple two-act story: Prelude to war, and th...",a simple twoact story prelude to war and the w...,simple twoact story prelude war war former con...,"[simple, twoact, story, prelude, war, war, for..."


In [36]:
# texto colocado stemming
from nltk.stem import SnowballStemmer

# 1. Crear un Stemmer para el texto
stemmer = SnowballStemmer(language='english')

# 2. Función para stemmizar tokens
def stemmizarTexto(tokens):
    return [stemmer.stem(token) for token in tokens]

In [37]:
df_critics['textoStemmizado'] = df_critics['textoTokenizado'].apply(stemmizarTexto)
df_critics.head()

,rotten_tomatoes_link,critic_name,top_critic,publisher_name,review_type,review_score,review_date,review_content,reviewsprepros,reviewssinstopwords,textoTokenizado,textoStemmizado
0,m/0814255,Andrew L. Urban,False,Urban Cinefile,Fresh,NaN,2010-02-06,A fantasy adventure that fuses Greek mythology...,a fantasy adventure that fuses greek mythology...,fantasy adventure fuses greek mythology contem...,"[fantasy, adventure, fuses, greek, mythology, ...","[fantasi, adventur, fuse, greek, mytholog, con..."
1,m/0814255,Louise Keller,False,Urban Cinefile,Fresh,NaN,2010-02-06,"Uma Thurman as Medusa, the gorgon with a coiff...",uma thurman as medusa the gorgon with a coiffu...,uma thurman medusa gorgon coiffure writhing sn...,"[uma, thurman, medusa, gorgon, coiffure, writh...","[uma, thurman, medusa, gorgon, coiffur, writh,..."
2,m/0814255,NaN,False,FILMINK (Australia),Fresh,NaN,2010-02-09,With a top-notch cast and dazzling special eff...,with a topnotch cast and dazzling special effe...,topnotch cast dazzling special effects tide te...,"[topnotch, cast, dazzling, special, effects, t...","[topnotch, cast, dazzl, special, effect, tide,..."
3,m/0814255,Ben McEachen,False,Sunday Mail (Australia),Fresh,3.5/5,2010-02-09,Whether audiences will get behind The Lightnin...,whether audiences will get behind the lightnin...,whether audiences get behind lightning thief h...,"[whether, audiences, get, behind, lightning, t...","[whether, audienc, get, behind, lightn, thief,..."
4,m/0814255,Ethan Alter,True,Hollywood Reporter,Rotten,NaN,2010-02-10,What's really lacking in The Lightning Thief i...,whats really lacking in the lightning thief is...,whats really lacking lightning thief genuine s...,"[whats, really, lacking, lightning, thief, gen...","[what, realli, lack, lightn, thief, genuin, se..."


In [38]:
df_critics.to_csv(r'archive\preprocesado.csv')

In [39]:
# Importar librerías necesarias
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Convertir la columna 'textoStemmizado' en texto plano para el cálculo de TF-IDF
# Cada documento debe ser una cadena de texto en vez de una lista de tokens
df_critics['textoStemmizadoPlano'] = df_critics['textoStemmizado'].apply(lambda x: ' '.join(x))


In [40]:

# Aplicar TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df_critics['textoStemmizadoPlano'])

# Guardar los nombres de las películas en un índice adicional
df_critics['movie_title'] = df_critics['rotten_tomatoes_link'].map(df_movies.set_index('rotten_tomatoes_link')['movie_title'])

In [41]:

# Función para simular una consulta y mostrar las películas más relevantes
def realizar_consulta(query, top_n=5):
    # Preprocesar la consulta igual que el resto de los textos
    query_preprocesada = preprocesamiento(query)
    query_sin_stopwords = eliminar_stopwords(query_preprocesada)
    query_stemmizada = ' '.join(stemmizarTexto(query_sin_stopwords.split()))

    # Convertir la consulta en un vector TF-IDF
    query_vector = vectorizer.transform([query_stemmizada])

    # Calcular la similitud coseno entre la consulta y los documentos
    similitudes = cosine_similarity(query_vector, tfidf_matrix).flatten()

    # Obtener los índices de las películas más relevantes
    indices_top = similitudes.argsort()[-top_n:][::-1]

    # Mostrar las películas más relevantes
    resultados = df_critics.iloc[indices_top][['movie_title']]
    resultados['score'] = similitudes[indices_top]
    return resultados

In [44]:
# Prueba de la simulación de consultas
consulta = "family movies"
resultados = realizar_consulta(consulta, top_n=5)
print("Resultados para la consulta:", consulta)
print(resultados)

Resultados para la consulta: family movies
               movie_title     score
494831      Kiss the Bride  0.866298
227611              Casper  0.821453
450712       Incredibles 2  0.795189
42378    Anna and the King  0.795189
1114171             Wonder  0.769107
